## Alabama Galaxy

Use this utlity to update the returns and std_dev fields within investment-options.csv

Globals

In [1]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True
throttle_limit=100
wait_time=30

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

In [4]:
# Read in investment options input csv.
df = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])
df

,Investment Option,Description,Asset Class,1 mo,3 mo,1 Yr,3 Yr,5 Yr,Vola,DS Vola,SD 1 Yr,SD 3 Yr,SD 5 Yr
0,ROAD,Construction Partners Inc,US Stocks:Industrials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EHC,Encompass Health,US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RF,Regions Financial,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VMC,Vulcan Materials Company,US Stocks:Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADTN,ADTRAN,US Stocks:Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PRA,ProAssurance Corporation,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TBRG,"TruBridge, Inc.",US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MPW,"Medical Properties Trust, Inc.",US Stocks:Real Estate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LAKE,"Lakeland Industries, Inc.",US Stocks:Consumer Staples,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [6]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries, throttle_limit=throttle_limit, wait_time=wait_time)
bb.compile_timeseries(symbols)

PRA VMC RF MPW LAKE EHC ADTN TBRG ROAD 


In [7]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,PRA,VMC,RF,MPW,LAKE,EHC,ADTN,TBRG,ROAD
Date,,,,,,,,,
2019-01-02,36.80,92.31,10.61,9.81,10.42,44.84,9.58,25.00,9.56
2019-01-03,36.43,91.36,10.59,10.11,10.32,44.04,9.61,24.87,9.66
2019-01-04,37.99,96.03,10.98,10.07,10.65,45.13,9.94,25.42,9.55
2019-01-07,37.84,97.94,11.17,10.24,10.91,46.25,10.19,25.81,9.75
2019-01-08,37.69,97.56,11.28,10.43,11.25,47.23,10.45,26.18,10.49
...,...,...,...,...,...,...,...,...,...
2025-04-28,23.12,245.39,20.46,5.50,16.59,115.32,7.80,25.44,80.60
2025-04-29,23.26,245.35,20.52,5.49,16.74,115.91,7.75,26.05,81.17
2025-04-30,23.21,262.33,20.41,5.52,16.33,116.99,7.66,26.03,82.14


In [8]:
# Calculate Annual Returns.
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

PRA     72.16
VMC      4.45
RF      14.04
MPW     23.12
LAKE     2.34
EHC     40.88
ADTN    79.91
TBRG   231.77
ROAD    73.45
dtype: float64

In [9]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [10]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [11]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [12]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [13]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [14]:
# Calculate 1 year, 3 year, and 5 year annualized standard deviation.
std_dev_1yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=1)
std_dev_3yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev_5yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=5)

In [15]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]

In [16]:
# For each symbol, output the performance metrics 
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd_1yr = std_dev_1yr[symbol]*100
    sd_3yr = std_dev_3yr[symbol]*100
    sd_5yr = std_dev_5yr[symbol]*100

    out.append((
        '{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}",'
        '"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
    ).format(
        fields[0], fields[1], fields[2], ret_1mo, ret_3mo, ret_1yr, ret_3yr,
        ret_5yr, _vola, _ds_vola, sd_1yr, sd_3yr, sd_5yr
    )) 

In [17]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')